In [1]:
import os
os.chdir('..')

In [2]:
from team_functions import *

In [3]:
options = Options()
options.add_argument('--headless')
driver = Firefox(options=options)

In [4]:
url = 'https://www.basketball-reference.com/boxscores/'
soup = getSoup(url, driver=driver)
driver.close()

In [5]:
summaries = soup.find('div', {'class': 'game_summaries'})
games = summaries.findAll('table', {'class': 'teams'})

games_df = []
for idx, game in enumerate(games):
    loser = game.find('tr', {'class': 'loser'}).find('td').find('a').get('href').split('/')[2]
    winner = game.find('tr', {'class': 'winner'}).find('td').find('a').get('href').split('/')[2]
    
    game_row = {'Winner': winner, 'Loser': loser}
    games_df.append(game_row)
    
games_df = pd.DataFrame(games_df)

In [6]:
games_df

,Winner,Loser
0,LAC,ATL
1,GSW,BRK
2,LAL,CHO
3,CLE,SAC
4,NOP,TOR
5,DAL,PHI


In [8]:
def checkPredictions(games_df, model_name='LR'):
    model_preds = pd.read_excel(f'Predictions/ModelsPredictions/{model_name}.xlsx')
    
    results = []
    for team in model_preds['visitor']:
        if team in list(games_df['Winner']):
            results.append(1)
        elif team in list(games_df['Loser']):
            results.append(0)
        else:
            print(f'Error checking results of team: {team}')

    model_preds['Results'] = np.array(results)
    model_preds['Checking'] = model_preds['Prediction'] == model_preds['Results']
    return model_preds